In [3]:
import os
import sys
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import logging

sys.path.append(os.pardir)
from database.database import Database
from config import SLACK_API_TOKEN, SLACK_CHANNEL, DATABASE_NAME
from utils.utilts import get_papers, fetch_interesting_points, fetch_summary

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()],
)

logger = logging.getLogger(__name__)


client = WebClient(token=SLACK_API_TOKEN)

In [4]:
db = Database(DATABASE_NAME)
db.init_database()

try:
    paper = get_papers(db)
    summary = fetch_summary(paper)
    interesting_points = fetch_interesting_points(paper)
    text = f"""
        *タイトル: {paper.title}*\n\n
        *著者名: {paper.authors}*\n\n
        *概要*\n{summary}\n\n
        *リンク*\n{paper.url}\n\n
        *提出日*\n{paper.submitted}\n\n
        *以下、面白いポイント*\n{interesting_points}\n\n
        ChatPDFで読む: https://www.chatpdf.com/ \n\n
        論文を読む: {paper.url}.pdf
    """
    try:
        response = client.chat_postMessage(channel=SLACK_CHANNEL, text=text)
    except SlackApiError as e:
        logger.error(f"Error posting to Slack: {e}")
    logger.info(f"Posted a paper: {paper.title}")
except Exception as e:
    logger.error(f"Error: {e}")

2025-04-03 23:08:37,010 [INFO] Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=%28ti%3A%22parallel+tempering%22+OR+ti%3A%22replica+exchange+monte+carlo%22%29+OR+%28au%3A%22Koji+Hukushima%22%29&id_list=&sortBy=submittedDate&sortOrder=descending&start=0&max_results=100
2025-04-03 23:08:38,623 [INFO] Got first page: 100 of 172 total results
2025-04-03 23:08:38,626 [INFO] INSERT : http://arxiv.org/abs/2409.07679v1 / Ratio Divergence Learning Using Target Energy in Restricted Boltzmann Machines: Beyond Kullback--Leibler Divergence Learning
2025-04-03 23:08:44,346 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-04-03 23:08:53,331 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-04-03 23:08:53,962 [INFO] Posted a paper: Ratio Divergence Learning Using Target Energy in Restricted Boltzmann Machines: Beyond Kullback--Leibler Divergence Learning
